In [171]:
# Imports
import pandas as pd

# MC expts.
def cam_revert_list():
    
    rl = {} # Revert List
        
    rl['MC-CAM6'] = ['f.e22.FHIST.f09_f09.cesm2_2.mcont_mjo.001']
    rl['MC-0x'] = ['f.e22.FHIST.f09_f09.cesm2_2.mcont_mjo_0x.001']
    rl['MC-2x'] = ['f.e22.FHIST.f09_f09.cesm2_2.mcont_mjo_2x.001']
    rl['MC-noMC'] = ['f.e22.FHIST.f09_f09.cesm2_2.mcont_mjo_ocn.001']
    rl['MC-noMC-c2sst'] = ['f.e22.FHIST.f09_f09.cesm2_2.mcont_mjo_ocn_c2sst.001']
    
    rl['MSMIP-c2msst'] = ['f.e22.FHIST.f09_f09.cesm2_2.MSMIP_cpl_msst.001']
    rl['MSMIP-c2dsst'] = ['f.e22.FHIST.f09_f09.cesm2_2.MSMIP_cpl_dsst.001']
    rl['MSMIP-c2csst'] = ['f.e22.FHIST.f09_f09.cesm2_2.MSMIP_cpl_clsst.001']
    
    rl['CESM2'] = ['b.e21.BHIST.f09_g17.CMIP6-historical.011']
   
    
    rl['GPCP'] = ['GPCP']
    rl['TRMM'] = ['TRMM']
    rl['ERAI'] = ['ERAI']
    rl['NVAP'] = ['NVAP']
    rl['MERRA'] = ['MERRA']
    
 # Data frame
    rl_df = pd.DataFrame.from_dict(rl, orient='index',columns=['run name'])
    return rl_df   
    

### MJO Maritime Continent Paper

02 Sep, 2022
- Read in climo files
- Mask islands regions in or out
- Average quantities over those regions
- Bar charts for different simulations

In [172]:
# Modules
import numpy as np
import importlib
import seaborn as sbn

import matplotlib.pyplot as mp
from matplotlib import cm
# My utils
import mcont_utils as mcont


In [179]:
# Settings

season = 'DJF'

var_name = 'LHFLX'
#var_name = 'TMQ'
#var_name = 'LHFLX'
var_name = "OMEGA500"

pref_out = 'MC_AMIP'
lrm_ave = True  ; fig_avdiff = 'diff'
#lrm_ave = False ; fig_avdiff = '_ave'

#case_desc =  np.array(['TRMM','GPCP','CESM2','MC-CAM6','MC-0x','MC-2x','MC-noMC'])
#case_lname = np.array(['TRMM','GPCP','CESM2','CAM6','0x orog','2x orog','No MC'])


case_desc =  np.array(['ERAI','CESM2','MC-CAM6','MC-0x','MC-2x','MC-noMC'])
case_lname = np.array(['ERAI','CESM2','CAM6','0x orog','2x orog','No MC'])


#case_desc =  np.array(['TRMM','GPCP','CESM2','MC-CAM6','MSMIP-c2msst','MSMIP-c2dsst','MSMIP-c2csst'])
#case_lname = np.array(['TRMM','GPCP','CESM2','CAM6','CESM2 monSST','CESM2 daySST','CESM2 climSST'])

lrm_case = True
land_frac_thresh = 0.5

In [180]:
# Loop

importlib.reload(mcont)

cases_df = cam_revert_list()
#var_mask_reg = pd.DataFrame(
#    columns=['Mask'],
#    index=case_lname)
var_mask_reg = pd.DataFrame()


#var_meta = get_var_meta(var_name)

for icase,case in enumerate(case_lname): # Loop cases

    cname = cases_df.loc[case_desc[icase]]['run name']
    clname = case_lname[icase]
    
    print('')
    print('++++++++++++ -> ')
    print('- Case',icase+1,' -> ',case_lname[icase],' ',cname)
    print('++++++++++++ -> ')
    print('')

# Case loop
        
    ds_in = mcont.read_climo(cname,var_name,season) # Grab dataset    

    var_mask_reg = mcont.reg_mask(ds_in,clname,var_name,var_mask_reg,land_frac_thresh=[0.5,0.55]) # Mask variable for orog,ocean,all single value)

    
# PLotting prep.
    
vunits = mcont.get_var_meta().loc[var_name]['units'] 
vlong_name = mcont.get_var_meta().loc[var_name]['Long Name'] 
tymin,tymax = ['avmin','avmax'] if lrm_ave else ['vmin','vmax'] 
ymin = mcont.get_var_meta().loc[var_name][tymin]  ; ymax = mcont.get_var_meta().loc[var_name][tymax] 


# Remove average of case 0?

for mname in ['Ocean','Land','Both']:

    mean_value = var_mask_reg.loc[var_mask_reg['Case'].isin([case_desc[0]]) & var_mask_reg['Mask'].isin([mname]),vunits].values
    var_mask_reg.loc[var_mask_reg['Mask'].isin([mname]),vunits] -= mean_value[0]
   
# Add back in to  
    var_mask_reg.loc[var_mask_reg['Case'].isin([case_desc[0]]) & var_mask_reg['Mask'].isin([mname]),vunits] = mean_value[0]
    
var_mask_reg



++++++++++++ -> 
- Case 1  ->  ERAI   ERAI
++++++++++++ -> 

-- Reading climo data in and constucting/reading land mask --
  - Reading in: /glade/p/cesm/amwg/amwg_data/obs_data/ERAI_DJF_climo.nc
-- Done --
-- Masking variable data --
-- Grabbing Data According to requested variable
OMEGA
-- Done --

++++++++++++ -> 
- Case 2  ->  CESM2   b.e21.BHIST.f09_g17.CMIP6-historical.011
++++++++++++ -> 

-- Reading climo data in and constucting/reading land mask --
  - Reading in: /glade/scratch/rneale/archive/b.e21.BHIST.f09_g17.CMIP6-historical.011/atm/climo/b.e21.BHIST.f09_g17.CMIP6-historical.011_DJF_climo.nc
-- Done --
-- Masking variable data --
-- Grabbing Data According to requested variable
OMEGA


KeyError: 500.0

In [ ]:
# PLOT #

mp.figure(figsize=(12,6))
print(var_mask_reg)
with sbn.axes_style("whitegrid"):
    splot = sbn.barplot(data=var_mask_reg,x='Case',y=vunits,hue='Mask')

splot.axes.vlines( x= 0.5,  color = "k", ls='--', lw = 3, ymin=ymin,ymax=ymax)
splot.axes.vlines( x= 2.5,  color = "k", ls='--', lw = 3, ymin=ymin,ymax=ymax)
mp.ylim(ymin,ymax)
splot.set(title=vlong_name)

sbn.set(rc={'axes.facecolor':'white','figure.facecolor':'white'}) 
mp.savefig(pref_out+'_'+var_name+'_'+fig_avdiff+'.png', dpi=120, bbox_inches='tight') 